# Visualization

In [1]:
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/MahalanobisAD/config.yaml'

## load data

In [7]:
machine_type = 'pump'
input_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
ext_data = pd.read_pickle(input_path)

In [19]:
pd.DataFrame(ext_data['valid_source']['features']).corr()

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,1.000000,0.699676,-0.270654,0.005829,-0.409808,0.907172,0.094111,0.005308,0.040821,0.023962,...,0.063928,0.082113,0.103108,0.044880,-0.121495,-0.267965,-0.022155,-0.089799,0.314820,0.079130
1,0.699676,1.000000,-0.159372,-0.078324,-0.383474,0.666501,0.127697,-0.031783,0.057963,0.048850,...,0.094330,0.187254,0.103477,0.043567,-0.216599,-0.292757,0.053952,-0.245510,0.485669,0.091504
2,-0.270654,-0.159372,1.000000,0.165675,0.207130,-0.106979,-0.000426,0.003378,0.016607,0.014681,...,0.000696,-0.010822,-0.009472,0.014795,0.086869,0.173041,0.012169,0.129038,-0.053815,0.022086
3,0.005829,-0.078324,0.165675,1.000000,0.816580,0.251890,0.610382,-0.066972,0.641189,0.630792,...,0.607176,0.476486,0.628622,0.669317,0.786182,0.671728,0.551894,0.772406,-0.503803,0.617533
4,-0.409808,-0.383474,0.207130,0.816580,1.000000,-0.154743,0.706422,-0.040327,0.756870,0.757405,...,0.721215,0.596234,0.720529,0.770111,0.870159,0.750706,0.720020,0.771514,-0.619774,0.693963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,-0.267965,-0.292757,0.173041,0.671728,0.750706,-0.090655,0.486567,0.008424,0.529757,0.528100,...,0.494855,0.323349,0.514463,0.569026,0.764971,1.000000,0.456649,0.822749,-0.538723,0.504552
3068,-0.022155,0.053952,0.012169,0.551894,0.720020,0.140123,0.985026,-0.046672,0.984153,0.986442,...,0.988461,0.914882,0.975635,0.967866,0.761661,0.456649,1.000000,0.508597,-0.304166,0.916079
3069,-0.089799,-0.245510,0.129038,0.772406,0.771514,0.074951,0.584249,0.004547,0.631563,0.626523,...,0.590014,0.328934,0.628718,0.688302,0.921494,0.822749,0.508597,1.000000,-0.783894,0.613875
3070,0.314820,0.485669,-0.053815,-0.503803,-0.619774,0.223752,-0.330530,-0.034963,-0.396296,-0.398188,...,-0.350157,0.022256,-0.381263,-0.451987,-0.782109,-0.538723,-0.304166,-0.783894,1.000000,-0.359858
